In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8461,2024-04-02,China Cba,08:35,Shanxi Zhongyu,Beijing,1.63,2.22,210.5,1.82,1.92,-3.5,1.98,2.77,2V3smZ5O,0.613497,0.450450,0.549451,0.520833,0.063947,0.8,0.4,NaN,NaN,288.732,103.179034,0.357352,231.016,68.706281,0.297409,234.654,290.300,234.90,208.82,0.0,0.0,0.0,0.0,0.216724,0.037813,0.235206,3.39,0.678,0.929204,0.561152,0.7,0.138848,2.34,0.468,2.606838,0.364035,0.3,-0.064035
8462,2024-04-02,Eua Nit,20:00,Indiana State,Utah Utes,1.65,2.33,163.5,1.91,1.91,-3.5,1.95,2.95,SI2ekaHC,0.606061,0.429185,0.523560,0.523560,0.035245,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,139.40,0.00,0.0,0.0,0.0,0.0,0.241624,0.000000,0.288615,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8463,2024-04-02,Lituânia Lkl,12:30,Jonava,Siauliai,1.62,2.20,168.5,1.85,1.85,-2.5,1.85,2.55,ttmGFhb7,0.617284,0.454545,0.540541,0.540541,0.071829,0.4,0.6,NaN,NaN,487.538,341.390178,0.700233,289.694,108.374477,0.374100,251.300,282.202,206.19,326.40,0.0,0.0,0.0,0.0,0.214724,0.000000,0.224989,-2.47,-0.494,-1.255061,0.000000,0.0,0.000000,0.68,0.136,8.823529,0.000000,0.0,0.000000
8464,2024-04-02,Suécia Superettan,15:00,Norrort,Sloga Uppsala,2.31,1.57,158.5,1.85,1.85,1.5,1.95,1.51,bZxikLah,0.432900,0.636943,0.540541,0.540541,0.069843,0.6,0.2,NaN,NaN,186.660,52.082135,0.279021,182.960,36.543287,0.199734,159.534,250.548,169.42,209.44,0.0,0.0,0.0,0.0,0.269721,0.000000,0.179842,-1.72,-0.344,-3.808140,0.000000,0.0,0.000000,2.11,0.422,1.350711,0.000000,0.0,0.000000


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,08:35,China Cba,Shanxi Zhongyu,Beijing,210.5,1.82,1.0,Over
1,20:00,Eua Nit,Indiana State,Utah Utes,163.5,1.91,1.0,Over
2,12:30,Lituânia Lkl,Jonava,Siauliai,168.5,1.85,1.0,Over
